%load_ext autoreload
%autoreload 2

In [104]:
import requests
import random
import json
import uuid
import numpy as np
from common.models.trial import Solution, Advisor, AdvisorSelection, WrittenStrategy, PostSurvey, Trial, TrialSaved, TrialError, SessionError
from utils.process import process_solution

In [105]:
# create new experiment

baseurl = "http://0.0.0.0:5050"

url = f"{baseurl}/admin/config"

experiment_type = 'sim_v1'

payload = json.dumps({
  "active": True,
  "created_at": "2023-10-17T09:57:36.204000",
  "redirect_url": "https://app.prolific.co/submissions/complete",
  "experiment_type": experiment_type,
  "rewrite_previous_data": True,
  "seed": 1,
  "n_generations": 5,
  "n_ai_players": 3,
  "networks_path": "data/23_11_30",
  "n_sessions_per_generation": 16,
  "n_advise_per_session": 5,
  "n_session_tree_replications": 1,
  "conditions": [
    "w_ai",
    "wo_ai"
  ],
  "n_social_learning_blocks": 1,
  "n_social_learning_networks_per_block": 4,
  "n_practice_trials": 2,
  "n_demonstration_trials": 4,
  "simulate_humans": False,
  "social_learning_trials": [
    "observation",
    "repeat",
    "try_yourself"
  ],
  "main_only": True
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic YWRtaW46YWRtaW4='
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"id":"65b18c53b19beea5c33c9b8e","active":true,"created_at":"2024-01-24T22:16:51.213352","redirect_url":"https://app.prolific.co/submissions/complete","experiment_type":"sim_v1","rewrite_previous_data":true,"networks_path":"data/23_11_30","seed":1,"n_generations":5,"n_ai_players":3,"n_sessions_per_generation":16,"n_advise_per_session":5,"n_session_tree_replications":1,"conditions":["w_ai","wo_ai"],"n_social_learning_blocks":1,"n_social_learning_networks_per_block":4,"n_practice_trials":2,"n_demonstration_trials":4,"simulate_humans":false,"social_learning_trials":["observation","repeat","try_yourself"],"main_only":true}


In [106]:
def get_trial(prolific_id, experiment_type):
    url = f"{baseurl}/session/{experiment_type}/{prolific_id}"
    payload={}
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    try:
      return Trial(**response.json())
    except:
      return None

def post_trial(prolific_id, trial_id, body):
    url = f"{baseurl}/session/{prolific_id}/{trial_id}"
    payload=body
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.status_code == 200

In [107]:
from pathlib import Path
from common.utils.utils import estimate_solution_score, estimate_average_player_score
from common.models.network import Network


networks_path = Path("../data/23_11_30")
network_data = json.load(open(networks_path / "networks.json"))
solutions_myopic = json.load(open(networks_path / "solution__myopic.json"))
solutions_m1 = json.load(open(networks_path / "solutions" / "0.json"))
networks_by_id = {n["network_id"]: n for n in network_data}
solutions_myopic_by_id = {s["network_id"]: s for s in solutions_myopic}
solutions_m1_by_id = {s["network_id"]: s for s in solutions_m1}


def _get_solution(network_id, solution_type):
    network = networks_by_id[network_id]
    # get the solution for the network
    if solution_type == "myopic":
        solution = solutions_myopic_by_id[network_id]
    elif solution_type == "machine":
        solution = solutions_m1_by_id[network_id]
    else:
        raise ValueError("Invalid solution type")

    solution['moves'][0] = network['starting_node']
    score = estimate_solution_score(Network(**network), solution['moves'], 10)
    assert score > 0, f"Invalid solution score: {score}"

    return Solution(**solution)


def get_solution(network_id, state):
    assert np.absolute(state.sum() - 1) < 0.0001, f"Invalid state: {state}"
    s_type_idx = np.random.choice(list(range(len(state))), p=state)
    s_type = ["myopic", "machine"][s_type_idx]
    return _get_solution(network_id, s_type)


def get_solution_evaluation(solution: Solution, network_id):
    # get rewards
    evaluation = process_solution(networks_by_id[network_id], solution.dict())
    return evaluation
    
    

In [108]:
social_learning_factor_obs = 0.0
social_learning_factor_rep = 0.5
individual_learning_factor = 0.01
individual_learning_factor_tys = 0.01


def handle_instruction_trial(trial, state):
    body = None
    return body, state

def handle_individual_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = np.array([0,1]) if random.random() < individual_learning_factor else state
    return solution.json(), state

def handle_written_strategy_trial(trial, state):
    strategy = WrittenStrategy(
        strategy=''
    )
    body = strategy.json()
    return body, state

def handle_demonstration_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    return solution.json(), state

def handle_debriefing_trial(trial, state):
    body = None
    return body, state

def handle_social_learning_selection_trial(trial, state):
    advisor_selection = trial.advisor_selection
    advisor = advisor_selection.advisor_ids
    scores = advisor_selection.scores
    
    # select the advisor with the highest score
    max_score_idx = np.argmax(scores)
    advisor = advisor[max_score_idx]
    
    
    selection = Advisor(
        advisor_id=advisor
    )
    body = selection.json()
    return body, state

def handle_observation_trial(trial, state):
    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    if evaluation['optimal'] == 10:
        state = np.array([0,1]) if random.random() < social_learning_factor_obs else state
    body = None
    return body, state

def handle_repeat_trial(trial, state):
    solution = trial.advisor.solution
    
    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    if evaluation['optimal'] == 10:
        state = np.array([0,1]) if random.random() < social_learning_factor_rep else state
    
    body = solution.json()
    return body, state

def handle_try_yourself_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = np.array([0,1]) if random.random() < individual_learning_factor_tys else state
    return solution.json(), state


def handle_trial(trial, state):
    if trial.trial_type == "instruction":
        return handle_instruction_trial(trial, state)
    elif trial.trial_type == "individual":
        return handle_individual_trial(trial, state)
    elif trial.trial_type == "written_strategy":
        return handle_written_strategy_trial(trial, state)
    elif trial.trial_type == "demonstration":
        return handle_demonstration_trial(trial, state)
    elif trial.trial_type == "debriefing":
        return handle_debriefing_trial(trial, state)
    elif trial.trial_type == "social_learning_selection":
        return handle_social_learning_selection_trial(trial, state)
    elif trial.trial_type == "observation":
        return handle_observation_trial(trial, state)
    elif trial.trial_type == "repeat":
        return handle_repeat_trial(trial, state)
    elif trial.trial_type == "try_yourself":
        return handle_try_yourself_trial(trial, state)
    else:
        raise ValueError(f"{trial.trial_type} is an invalid trial type")

In [109]:
trials = []

finished = False
while not finished:
    prolific_id = "sim_" + uuid.uuid4().hex[:8]
    state = np.array([1, 0]) # [myopic, machine]
    current_trial_id = None
    while True:
        trial = get_trial(prolific_id, experiment_type)
        if trial is None:
            finished = True
            break
        if trial.trial_type in ['individual', 'observation', 'repeat', 'try_yourself']:
            print(f'ProlificId: {prolific_id}, TrialId: {trial.id}, TrialType: {trial.trial_type} {state}')
        if trial.id == current_trial_id:
            break
        current_trial_id = trial.id
        body, state = handle_trial(trial, state)
        post_trial(prolific_id, trial.id, body)
        trial_clean = json.loads(trial.json())
        # session_clean = json.loads(trail.session.json())
        trials.append({'trial': trial_clean, 'prolific_id': prolific_id})



# save trials as json
import json
with open('trials.json', 'w') as f:
    json.dump(trials, f, indent=4)

ProlificId: sim_d65d2b45, TrialId: 1, TrialType: individual [1 0]
ProlificId: sim_d65d2b45, TrialId: 2, TrialType: individual [1 0]
ProlificId: sim_d65d2b45, TrialId: 3, TrialType: individual [1 0]
ProlificId: sim_d65d2b45, TrialId: 4, TrialType: individual [1 0]
ProlificId: sim_51256eb9, TrialId: 1, TrialType: individual [1 0]
ProlificId: sim_51256eb9, TrialId: 2, TrialType: individual [1 0]
ProlificId: sim_51256eb9, TrialId: 3, TrialType: individual [1 0]
ProlificId: sim_51256eb9, TrialId: 4, TrialType: individual [1 0]
ProlificId: sim_8a560249, TrialId: 1, TrialType: individual [1 0]
ProlificId: sim_8a560249, TrialId: 2, TrialType: individual [1 0]
ProlificId: sim_8a560249, TrialId: 3, TrialType: individual [1 0]
ProlificId: sim_8a560249, TrialId: 4, TrialType: individual [1 0]
ProlificId: sim_0c953f34, TrialId: 1, TrialType: individual [1 0]
ProlificId: sim_0c953f34, TrialId: 2, TrialType: individual [1 0]
ProlificId: sim_0c953f34, TrialId: 3, TrialType: individual [1 0]
ProlificId

In [110]:
trial_clean

{'id': 21,
 'trial_type': 'debriefing',
 'instruction_type': None,
 'finished': False,
 'started_at': '2024-01-24T22:21:43.288283',
 'finished_at': None,
 'network': None,
 'solution': None,
 'advisor': None,
 'advisor_selection': None,
 'selected_by_children': [],
 'written_strategy': None,
 'post_survey': None,
 'redirect_url': 'https://app.prolific.co/submissions/complete',
 'is_practice': False,
 'trial_title': '',
 'last_trial_for_current_example': False,
 'social_learning_block_idx': 0,
 'block_network_idx': 0}